# Importação das bibliotecas básicas

In [ ]:
!pip install plotly --upgrade

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import accuracy_score, classification_report

## Base risco de crédito

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_risco_credito = pd.read_csv('/content/drive/MyDrive/ic/risco_credito.csv')

In [ ]:
base_risco_credito

In [ ]:
X_risco_credito = base_risco_credito.iloc[:, 0:4].values
X_risco_credito

In [ ]:
y_risco_credito = base_risco_credito.iloc[:, 4].values
y_risco_credito

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoders = LabelEncoder()
for i in range(0,4):
  X_risco_credito[:,i] = label_encoders.fit_transform(X_risco_credito[:,i])

# Árvores de decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier,export_graphviz

## Base risco de crédito

In [ ]:
X_risco_credito

In [ ]:
y_risco_credito

In [ ]:
arvore_risco_credito = DecisionTreeClassifier(criterion='entropy',max_depth = 3)
arvore_risco_credito.fit(X_risco_credito, y_risco_credito)

In [ ]:
arvore_risco_credito.feature_importances_

In [ ]:
arvore_risco_credito.classes_

In [ ]:
from sklearn import tree
previsores = ['história', 'dívida', 'garantias', 'renda']
figura, eixos = plt.subplots(nrows=1, ncols=1, figsize=(10,10))
tree.plot_tree(arvore_risco_credito, feature_names=previsores, class_names = arvore_risco_credito.classes_, filled=True);

In [ ]:
# história boa, dívida alta, garantias nenhuma, renda > 35
# história ruim, dívida alta, garantias adequada, renda < 15
previsoes = arvore_risco_credito.predict([[0,0,1,2],[2,0,0,0]])
previsoes

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import SVG,display
from graphviz import Source

In [ ]:
def plot_tree(crit, split, depth, min_samples_split, min_samples_leaf=0.2):
    estimator = DecisionTreeClassifier(
           random_state = 0 
          ,criterion = crit
          ,splitter = split
          ,max_depth = depth
          ,min_samples_split=min_samples_split
          ,min_samples_leaf=min_samples_leaf
    )
    estimator.fit(X_risco_credito, y_risco_credito)
    graph = Source(export_graphviz(estimator
      , out_file=None
      , feature_names=previsores
      , class_names=arvore_risco_credito.classes_
      , impurity=True
      , filled = True))
    display(SVG(graph.pipe(format='svg')))
    return estimator

In [ ]:
interact(plot_tree 
   , crit = ["gini", "entropy"]
   , split = ["best", "random"]
   , depth=[1,2,3,4,5,10,20,30]
   , min_samples_split=(1,5)
   , min_samples_leaf=(1,5))

## Base credit data - 98.20%

In [ ]:
import pickle
with open('/content/drive/MyDrive/ic/credit.pkl', 'rb') as f:  
  X_credit_treinamento, y_credit_treinamento, X_credit_teste, y_credit_teste = pickle.load(f)

In [ ]:
X_credit_treinamento.shape, y_credit_treinamento.shape

In [ ]:
X_credit_teste.shape, y_credit_teste.shape

In [ ]:
arvore_credit = DecisionTreeClassifier(criterion='entropy', random_state = 0)
arvore_credit.fit(X_credit_treinamento, y_credit_treinamento)

In [ ]:
previsoes = arvore_credit.predict(X_credit_teste)
previsoes

In [ ]:
y_credit_teste

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score(y_credit_teste, previsoes)

In [ ]:
from yellowbrick.classifier import ConfusionMatrix
cm = ConfusionMatrix(arvore_credit)
cm.fit(X_credit_treinamento, y_credit_treinamento)
cm.score(X_credit_teste, y_credit_teste)

In [ ]:
print(classification_report(y_credit_teste, previsoes))

In [ ]:
arvore_credit.classes_

In [ ]:
from sklearn import tree
previsores = ['income', 'age', 'loan']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (20,20))
tree.plot_tree(arvore_credit, feature_names=previsores, class_names=['0','1'], filled=True);
fig.savefig('arvore_credit.png')